<a href="https://colab.research.google.com/github/sarahsultan/joelnet/blob/master/longformer_document_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

In [3]:
!mkdir folderOnColab
!gcsfuse rokin-articles folderOnColab

2021/06/01 10:30:23.147315 Using mount point: /content/folderOnColab
2021/06/01 10:30:23.155198 Opening GCS connection...
2021/06/01 10:30:23.701073 Mounting file system "rokin-articles"...
2021/06/01 10:30:23.741715 File system has been successfully mounted.


In [4]:
import pandas as pd
cleaned = pd.read_json("/content/folderOnColab/from_2017_dataset_english.json.gz",
                       compression="gzip",
                       orient="records",
                       lines=True)
cleaned = cleaned.values.flatten()
cleaned = cleaned[cleaned!=None]
cleaned = pd.DataFrame.from_records(cleaned)

In [5]:
# remove articles of nature.com
cleaned = cleaned[cleaned.sitename != 'nature.com']

In [6]:
# articles text in a list
article_text  = []  # each item of this list is a string which is one article
for index, row in cleaned.iterrows():
    sample_txt = row['text']
    article_text.append(sample_txt)

In [7]:
len(article_text)

554020

In [ ]:
pip install transformers

In [ ]:
pip install transformers faiss torch

In [ ]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

In [ ]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
model = AutoModel.from_pretrained("allenai/longformer-base-4096")

In [12]:
documents = article_text[0:110]

vectors = [
  # tokenize the document, return it as PyTorch tensors (vectors),
  # and pass it onto the model
  model(**tokenizer(document, return_tensors='pt'))[0].detach().squeeze()
  for document in documents
]

In [ ]:
import torch

averaged_vectors = [torch.mean(vector, dim=0) for vector in vectors]

[v.size() for v in averaged_vectors]

In [14]:
def encode(document: str) -> torch.Tensor:
  tokens = tokenizer(document, return_tensors='pt')
  vector = model(**tokens)[0].detach().squeeze()
  return torch.mean(vector, dim=0)


In [15]:
import faiss
import numpy as np

index = faiss.IndexIDMap(faiss.IndexFlatIP(768)) # the size of our vector space
# index all the documents, we need them as numpy arrays first
index.add_with_ids(
    np.array([t.numpy() for t in averaged_vectors]),
    # the IDs will be 0 to len(documents)
    np.array(range(0, len(documents)))
)

def search(query: str, k=1):
  encoded_query = encode(query).unsqueeze(dim=0).numpy()
  top_k = index.search(encoded_query, k)
  scores = top_k[0][0]
  results = [documents[_id] for _id in top_k[1][0]]
  return list(zip(results, scores))

UsageError: Line magic function `%install_ext` not found.


In [ ]:
%time 
search("virtual reality", k=5)